In [4]:
'Hello World!'

'Hello World!'

In [5]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

2.9.0+cu126
True
1
0
Tesla T4


In [6]:
import os
import math
import random
from glob import glob
import os
import kagglehub
import shutil

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"

In [9]:
os.environ["KAGGLE_USERNAME"] = "tejaskumarvurs"
os.environ["KAGGLE_KEY"] = "80fbc5d540819df3b4666ae5df969af9"

path = kagglehub.dataset_download("tejaskumarvurs/gen-ai-animal-dataset")

dest = "/content/animal_data"
if not os.path.exists(dest):
    shutil.copytree(path, dest)

print(f"✅ Dataset is ready at: {dest}")

Using Colab cache for faster access to the 'gen-ai-animal-dataset' dataset.
✅ Dataset is ready at: /content/animal_data


In [10]:
!find / -name "gen-ai-animal-dataset" -type d 2>/dev/null

/kaggle/input/gen-ai-animal-dataset


In [11]:
# Force a link from the hidden system path to your visible content folder
!ln -s /root/.cache/kagglehub/datasets/tejaskumarvurs/gen-ai-animal-dataset/versions/1 /content/dataset

# Now check if Colab can 'see' into that shortcut
!ls /content/dataset | head -n 5

!ls /content/dataset/Camel | head -n 5

ln: failed to create symbolic link '/content/dataset': File exists
/content/dataset
ls: cannot access '/content/dataset/Camel': No such file or directory


In [9]:
DATA_PATH = "/content/dataset"

def count_files(directory):
    return sum([len(files) for r, d, files in os.walk(directory)])

print(f"Total files accessible for T4 training: {count_files(DATA_PATH)}")

Total files accessible for T4 training: 29071


In [10]:
path = "/content/dataset"

if os.path.exists(path):
    contents = os.listdir(path)
    print(f"✅ Path exists!")
    print(f"Items inside '{path}': {contents[:5]}") # Shows first 5 items
    
    # Check if the first item is a file or a folder
    if len(contents) > 0:
        first_item = os.path.join(path, contents[0])
        if os.path.isdir(first_item):
            print("📁 Found subfolders (Classes). Use datasets.ImageFolder(path)")
        else:
            print("🖼️ Found direct files. Use a custom Dataset class.")
else:
    print(f"❌ Path NOT found: {path}")
    print("Checking /content/ to see what is actually there:")
    print(os.listdir("/content/"))

✅ Path exists!
Items inside '/content/dataset': ['Deer', 'Panda', 'Turkey', 'Goat', 'Canary']
📁 Found subfolders (Classes). Use datasets.ImageFolder(path)


In [12]:
class ImageDataset(Dataset):
    def __init__(self, root, size=64):
        # Fixed the typo: "**/*.jpg" instead of "**/*,jpg"
        self.files = glob(os.path.join(root, "**/*.jpg"), recursive=True)
        
        if len(self.files) == 0:
            print(f"⚠️ Warning: No .jpg files found in {root}. Check path or extensions.")
            
        self.transform = transforms.Compose([
            transforms.Resize(size),
            transforms.CenterCrop(size),
            transforms.ToTensor(),
            transforms.Normalize([0.5]*3, [0.5]*3) # Scales to [-1, 1]
        ])
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        img = Image.open(self.files[idx]).convert("RGB")
        return self.transform(img)

In [14]:
# Print the first 20 file paths to see if they look correct
test_ds = ImageDataset("/content/dataset")
print(test_ds.files[:20])

['/content/dataset/Deer/4fe91d2f25f3b6a5.jpg', '/content/dataset/Deer/511aa4fa584803b3.jpg', '/content/dataset/Deer/a760f2fee588211e.jpg', '/content/dataset/Deer/bba443efc42d285e.jpg', '/content/dataset/Deer/f73679d797ac75a8.jpg', '/content/dataset/Deer/5eef9054a0123096.jpg', '/content/dataset/Deer/5c52a9fc6ec207f1.jpg', '/content/dataset/Deer/2455b8585d5bf0d9.jpg', '/content/dataset/Deer/1fdb9545a37ae8a9.jpg', '/content/dataset/Deer/09ab94e77f37e1c1.jpg', '/content/dataset/Deer/c65823cfb39ef048.jpg', '/content/dataset/Deer/027928cdac94d62d.jpg', '/content/dataset/Deer/7e7c478ee979f3c2.jpg', '/content/dataset/Deer/e1213ee2ce7c3e9b.jpg', '/content/dataset/Deer/8cd7ebd5d9ecde8f.jpg', '/content/dataset/Deer/99606eaef7120ad8.jpg', '/content/dataset/Deer/dc60f8e75d4e4668.jpg', '/content/dataset/Deer/a966b561b9ff2ad1.jpg', '/content/dataset/Deer/d878d864f26a8e67.jpg', '/content/dataset/Deer/71c05c33211b1416.jpg']


In [15]:
class MappingNetwork(nn.Module):
    def __init__(self, z_dim=512, w_dim=512):
        super().__init__()
        layers = []
        for _ in range(8):
            layers.append(nn.Linear(w_dim, w_dim))
            layers.append(nn.LeakyReLU())
        self.mapping = nn.Sequential(*layers)
    
    def forward(self, z):
        z = z / z.norm(dim=1, keepdim=True)
        return self.mapping(z)

In [16]:
class StyleConv(nn.Module):
    def __init__(self, in_ch, out_ch, w_dim):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(out_ch, in_ch, 3, 3))
        self.style = nn.Linear(w_dim, in_ch)
        self.noise_strength = nn.Parameter(torch.zeros(1))
        self.bias = nn.Parameter(torch.zeros(out_ch))
    
    def forward(self, x, w, noise):
        b, c, h, w_ = x.shape
        style = self.style(w).view(b, 1, c, 1, 1)
        weight = self.weight.unsqueeze(0) * style
        weight = weight.view(-1, c, 3, 3)

        x = x.view(1, -1, h, w_)
        x= F.conv2d(x, weight, padding=1, groups=b)
        x = x.view(b, -1, h, w_)

        x = x + self.noise_strength * noise
        return x + self.bias.view(1, -1, 1, 1)

In [1]:
class Generator(nn.Module):
    def __init__(self, z_dim=512, w_dim=512):
        super().__init__()
        self.mapping = MappingNetwork(z_dim, w_dim)

        self.const = nn.Parameter(torch.randn(1, 512, 4, 4))

        self.layers = nn.ModuleList([
            StyleConv(512, 512, w_dim),
            StyleConv(512, 256, w_dim),
            StyleConv(256, 128, w_dim),
            StyleConv(128, 64, w_dim),
          ])
        
        self.to_rgb = nn.Conv2d(64, 3, 1)

    def forward(self, z):
        w = self.mapping(z)
        x = self.const.repeat(z.size(0), 1, 1, 1)

        for layer in self.layers:
            x = F.interpolate(x, scale_factor=2, mode='bilinear', align_corners=False)

            noise = torch.randn(x.size(0), 1, x.size(2), x.size(3), device=x.device)

            x = layer(x, w, noise)
            x = F.leaky_relu(x, 0.2)

        return torch.tanh(self.to_rgb(x))

NameError: name 'nn' is not defined

In [18]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        def block(in_c, out_c):
            return nn.Sequential(
                nn.Conv2d(in_c, out_c, 4, 2, 1),
                nn.LeakyReLU(0.2)
            )
        
        self.net = nn.Sequential(
            block(3, 64),
            block(64, 128),
            block(128, 256),
            block(256, 512),
            nn.Flatten(),
            nn.Linear(512*4*4, 1)
        )

    def forward(self, x):
        return self.net(x)


In [19]:
def Gradient_Penalty(D, real, fake):
    alpha = torch.rand(real.size(0), 1, 1, 1).to(device)
    interp = (alpha * real + (1 - alpha) * fake).requires_grad_(True)
    out = D(interp)

    grads = torch.autograd.grad(
        outputs=out,
        inputs=interp,
        grad_outputs=torch.ones_like(out),
        create_graph=True,
    )[0]

    return ((grads.norm(2, dim=1) - 1) ** 2).mean()

In [2]:
def train():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    path = "/content/dataset"  # Update this path as needed
    dataset = ImageDataset(path, size=64)
    print(f"✅ Dataset loaded with {len(dataset)} images.")
    loader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)

    G = Generator().to(device)
    D = Discriminator().to(device)

    g_opt = torch.optim.Adam(G.parameters(), lr=1e-4, betas=(0.0, 0.99))
    d_opt = torch.optim.Adam(D.parameters(), lr=1e-4, betas=(0.0, 0.99))

    for epoch in range(50):
        for i, real in enumerate(loader):
            real = real.to(device)
            z = torch.randn(real.size(0), 512).to(device)
            fake = G(z)

            d_loss = D(fake).mean() - D(real).mean()
            gp = Gradient_Penalty(D, real, fake)
            d_total = d_loss + 10 * gp

            d_opt.zero_grad()
            d_total.backward()
            d_opt.step()

            if i % 5 == 0:
                g_loss = -D(G(z)).mean()
                g_opt.zero_grad()
                g_loss.backward()
                g_opt.step()

        with torch.no_grad():
            samples = G(torch.randn(16, 512).to(device))
            save_image(samples, f"samples/epoch_{epoch}.png", normalize=True)

        print(f"Epoch {epoch} | D: {d_total.item():.3f} | G: {g_loss.item():.3f}")

In [3]:
if __name__ == "__main__":
    train()

NameError: name 'torch' is not defined